<a href="https://colab.research.google.com/github/BeatriceZbj/MachineLearningAlgorithm/blob/main/Exploratory_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('air_data.csv') #读取原始数据
data

In [ ]:
data.head() #tail

In [ ]:
data.shape

In [ ]:
data.info()     #缺失情况，数据类型

#### 缺失值计算

In [ ]:
# 对数据的描述，探索性分析
explore = data.describe().T    #数值型
explore

#### 数值型

In [ ]:
#哪些列有缺失，缺失的数据类型，缺失的数量，缺失的比例
explore['null'] = len(data)-explore['count'] # describe()函数自动计算非空值数，需要手动计算空值数
explore = explore[['null', 'max', 'min']]
explore.columns = ['空值数', '最大值', '最小值'] # 表头重命名
explore['空值比例'] =explore['空值数']/len(data)
explore  

In [ ]:
explore[explore['空值数']>0]  #数值型数据，筛选有空值

#### 字符串型

In [ ]:
explore2=data.describe(include='object').T    #类别型 include='object'
explore2

In [ ]:
explore2['null'] = len(data)-explore2['count'] # describe()函数自动计算非空值数，需要手动计算空值数
explore2['空值比例'] =explore2['null']/len(data)
explore2

In [ ]:
explore2[explore2['null']>0]  #对于object型，空值情况

#### 数据预处理

In [ ]:
# 处理方式，过滤：
# 票价为空的记录（第一年和第二年票价都不能为空）：为空可能是不存在的飞行记录

##### 数据清洗

In [ ]:
# 过滤掉票价为空的记录，两年都不为空的数据
data = data[data['SUM_YR_1'].notnull() & data['SUM_YR_2'].notnull()] 

In [ ]:
data.shape

In [ ]:
# 选出有乘机记录的乘客选出来，乘坐过飞机的客户，三类
#1、要么第一年的总票价不为0
#2、要么第二年的总票价不为0
#3、积累的里程为0，同时折扣率也为0，积分兑换用户

index1 = data['SUM_YR_1'] != 0  #第一年不等于零 选出
index2 = data['SUM_YR_2'] != 0  #第二年不等于零  选出
index3 = (data['SEG_KM_SUM'] == 0) & (data['avg_discount'] == 0)  # 积分兑换用户 
data = data[index1 | index2 | index3] # 逻辑或  用户的筛选
data.shape

##### 字段选择

In [ ]:
# LOAD_TIME：数据截取时间（观察期的结束时间，这里为2014-2-31）
# FFP_DATE：入会时间              —— L
# LAST_TO_END：最后一次乘机时间   —— R
# FLIGHT_COUNT：飞行次数  —— F
# SEG_KM_SUM：飞行总里程  —— M
# avg_discount ：平均折扣率 —— C

#构造LRFMC指标：
#L=LOAD_TIME - FFP_DATE(观测窗口的结束时间 - 入会时间)  # 入会时长，时间间隔
#R=LAST_TO_END(最后一次乘坐飞机距观测窗口结束的时长)    #时间间隔
#F=FLIGHT_COUNT(观测窗口内的飞行次数)
#M=SEG_KM_SUM(观测窗口内的总飞行里程)
#C=AVG_DISCOUNT(平均折扣率)

In [ ]:
data = data[['LOAD_TIME','FFP_DATE','LAST_TO_END','FLIGHT_COUNT','SEG_KM_SUM','avg_discount']]
data.head()

In [ ]:
data.info()

##### 时间数据处理

In [ ]:
pd.to_datetime(data['LOAD_TIME'])- pd.to_datetime(data['FFP_DATE']) 

In [ ]:
L = pd.to_datetime(data['LOAD_TIME'])- pd.to_datetime(data['FFP_DATE']) 
data['L'] = L.apply(lambda x:x.days/30)  #入会时长
data['R'] = data['LAST_TO_END']
data['F'] = data['FLIGHT_COUNT']
data['M'] = data['SEG_KM_SUM']
data['C'] = data['avg_discount']
data1 = data[['L','R','F','M','C']]
data1.head()

##### 无量钢化

In [ ]:
# 5个指标的取值范围数据差异较大，为了消除数量级数据带来的影响，对数据进行标准化处理。
explore1 = data1.describe().T
explore1

In [ ]:
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
data_sta=pd.DataFrame(std.fit_transform(data1))

In [ ]:
# 标准差标准化
data_sta.columns=['Z'+i for i in data1.columns] # 表头重命名 表示经过标准化的数据
data_sta.head()